# Make requests to OpenAI ChatCompletion API

input: `order_file_path`

output: `api_request_file_path`

Description
- `order_file_path` contains `Platform A`'s order-requst data on a day. 
- This notebook make a request to OpenAI ChatCompletion API with this data.
- Based on the Prompt Template, it writes a JSON file that contains the request data.

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import re

You should specify the file path which contains your openai API key.

In [2]:
import openai
import os
with open("../local_data/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss

In [3]:
from openai import OpenAI
client = OpenAI()

*REMARK:*

We provide only a sample of 10,000 order_item_descriptions that were collected by Platform A on a specific date. 

Please understand that we cannot disclose the entire dataset as it is a proprietary asset of the company.

In [5]:
engine_id = 'gpt-4-turbo'
N=100
dataset_size = 40000
order_file_path = r'../local_data/order_item_description_samples.csv'
api_request_file_path = r'../local_data/api_requests_for_annotated_dataset.jsonl'

In [5]:
order_df = pd.read_csv(order_file_path, sep='\t')
order_names = pd.Series(order_df.goods_name.value_counts().index).rename('sentence')
print(f"num examples:{len(order_names)} before filtering")
len_mean = order_names.str.len().mean()
len_stdev = order_names.str.len().std()
order_names = order_names[order_names.str.len() <= 100].sample(frac=1, random_state=42).reset_index(drop=True)
print(f"num examples:{len(order_names)}, length's mean: {len_mean}, length's stdev: {len_stdev} after filtering")

num examples:137289 before filtering
num examples:136176, length's mean: 14.785168513136522, length's stdev: 203.01567758921644 after filtering


In [6]:
def remove_non_printable_chars(s):
    # remove non-printable unicode characters.
    return re.sub(r'[^\x20-\x7E\u00A0-\uD7FF\uE000-\uFFFD]', '', s)

In [7]:
order_names = order_names.apply(remove_non_printable_chars)
order_names = order_names[:dataset_size]

# Prompt Template

In [28]:
system_prompt = """너는 입력 주문 상품명으로부터 실제 상품명만 추출하는 역할을 수행한다.
이벤트 문구가 추가된 경우: '행사중)', '!후기이벤트!' 같은 이벤트 문구가 본래의 실제 상품명 왼쪽 오른쪽에 붙은 경우가 있어. 이런 이벤트 문구를 제거해주고 실제 상품명만 추출해야 한다.
- 예: 행사중)닭강정 도시락 !리뷰이벤트! -> 닭강정 도시락
- 예: [오늘만 할인]고르곤졸라 피자+콜라1.5L -> 고르곤졸라 피자+콜라1.5L

수식어 문구가 추가된 경우: 관용구, 이모티콘, 형용사등 수식어가 본래의 실제 상품명 왼쪽 오른쪽에 붙은 경우가 있어. 이런 수식어를 제거해주고 실제 상품명만 추출해야 한다.
- 예: *강북1위*진짜 돼지국밥 (국물이 끝내줘요 ~) -> 진짜 돼지국밥
- 예: 무더위엔***수삼 반계탕 $구수합니다$ -> 수삼 반계탕

상품명이 아닌 표현: 실제 상품명이 없는 것을 판단되면 빈 문자열을 반환해야 한다.
- 예: 배달팁 -> ''
- 예: 숟가락 주지 마세요 -> ''

주의사항: 문장 오른쪽에 붙는 옵션명이나 수량은 유지해 주어야 한다.
- 예: + 프렌치 프라이(L) -> 프렌치 프라이(L) 
- 예: 떡볶이(소) -> 떡볶이(소)

### 몇가지 바람직한 실제 상품명 추출 예시를 나열할테니 이를 참고해주길 바란다.  
[New]SG크림치즈버거(L) -> SG크림치즈버거(L)
(BEST)묵은지 김치찌개 -> 묵은지 김치찌개
(반반)[인기]에그콘피자'-> (반반)[인기]에그콘피자
바베큐치킨 (반) -> 바베큐치킨 (반)
밥 300g 으로 변경 -> 밥 300g
문어볼 2알 추가 -> 문어볼 2알
김치 찐 만두(6개) -> 김치 찐 만두(6개)
청경채 추가 -> 청경채 추가
중간맛 ( 불닭 볶음면 정도 ) -> 중간맛
맛집랭킹[낙.곱.새한우대창]조리후 배달 -> 낙.곱.새한우대창
[알싸씨크개성강한] 에그 마늘쫑 김밥 -> 에그 마늘쫑 김밥
푸짐한 손만두 김치찌개 -> 손만두 김치찌개
몬스터 포테이토 피자 -> 몬스터 포테이토 피자
메인 두 그릇+탕수육 2인세트 -> 메인 두 그릇+탕수육 2인세트
자부심초밥 세트(10P) -> 자부심초밥 세트(10p)
올데이프룻자두 500ml -> 올데이프룻자두 500ml
!R4 파인애플 볶음밥 -> 파인애플 볶음밥
치즈프로마쥬 흑당밀크티(펄 기본 제공)(ICE(대용량)) -> 치즈프로마쥬 흑당밀크티(펄 기본 제공)(ICE(대용량))
더블 콤보 에그누들 볶음면 -> 더블 콤보 에그누들 볶음면
부추2개(2개) -> 부추2개(2개)
스프라이트 1.25L 추가 -> 스프라이트 1.25L
숯불직화삼겹모듬(2인) -> 숯불직화삼겹모듬(2인)
에그샐러드 샌드위치 -> 에그샐러드 샌드위치
불야채막창+체다소스+쿨피스450ml+각무 -> 불야채막창+체다소스+쿨피스450ml+각무
김추가(김10개+마니) -> 김추가
순대국밥 000032 -> 순대국밥
체다 스커트버거 -> 체다 스커트버거
[든든한]미숫가루(500ml) -> 미숫가루(500ml)
생선카츠(1조각) -> 생선카츠(1조각)
의성마늘찜닭(중) -> 의성마늘찜닭(중)
세지멜론 -> 세지멜론
생과일아이스샤인탕후루 남은수량 : 1개 -> 생과일아이스샤인탕후루
소 (2인 ) -> 소 (2인)
!오늘의 초밥 10p -> 오늘의 초밥 10p
딥초코가나슈마카롱~ -> 딥초코가나슈마카롱
브로콜리 8조각 -> 브로콜리 8조각
콩나물제육볶음(2인분) -> 콩나물제육볶음(2인분)
볶음) 푸주추가 (75g) -> 푸주추가 (75g)
[RI뷰E벤트] 선택 안함 -> ''
"""

In [29]:
user_prompt = """### 다음 각 주문 상품명을 분석해서 예시처럼 -> 로 구분하여 실제 상품명을 출력해줘. 반드시 주문 상품명은 그대로 출력해줘.
{0}
"""

# Asynchronous Request Preparation

In [40]:
def make_json_body(request_id, system_prompt, user_prompt):
    dic = { 
        'model':engine_id,
        'messages':[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        'temperature':0.0,
        'metadata':{'request_id':request_id},
    }
    return json.dumps(dic, ensure_ascii=False)

In [41]:
with open(api_request_file_path, 'w') as f:
    for i in range(0, len(order_names), N):
        jsonl = make_json_body(
            request_id=i,
            system_prompt=system_prompt,
            user_prompt=user_prompt.format('\n'.join(order_names[i:i+N].values))
            )      
        f.write(jsonl + '\n')